In [128]:
from requests import get
import json
import pandas as pd
import numpy as np
import pickle

In [21]:
# Arbejder med JP's (JP/Politiken?) data

In [22]:
"""
compareCandidateUserUrl: "https://kit.jyllands-posten.dk/electionendpoint/candidate/getUserResult/"
comparePartyUserUrl: "https://kit.jyllands-posten.dk/electionendpoint/party/getUserResult/"
defaultDistrictID: "49"
electionsId: "28"
getCandidateBasicUrl: "https://kit.jyllands-posten.dk/electionendpoint/candidate/getCandidate/"
getCandidateMatchesBasicUrl: "https://kit.jyllands-posten.dk/electionendpoint/candidate/getUserMatches/"
getCandidateMatchesWithCandidateUrl: "https://kit.jyllands-posten.dk/electionendpoint/candidate/getCandidateMatches/"
getCandidateMatchesWithPartyUrl: "https://kit.jyllands-posten.dk/electionendpoint/party/getCandidateMatches/"
getCandidatePollResult: "https://kit.jyllands-posten.dk/electionendpoint/candidate/getCandidateResult/"
getCandidatesByDistrictBasicUrl: "https://kit.jyllands-posten.dk/electionendpoint/district/getCandidates/"
getCandidatesByPartyAndDistrictUrl: "https://kit.jyllands-posten.dk/electionendpoint/candidate/getCandidates/"
getCandidatesByPartyIdUrl: "https://kit.jyllands-posten.dk/electionendpoint/candidate/getCandidates/"
getDistrictsBasicUrl: "https://kit.jyllands-posten.dk/electionendpoint/district/getDistricts/"
getPartiesBasicUrl: "https://kit.jyllands-posten.dk/electionendpoint/party/getParties/"
getPartiesByDistrictBasicUrl: "https://kit.jyllands-posten.dk/electionendpoint/district/getParties/"
getPartyBasicUrl: "https://kit.jyllands-posten.dk/electionendpoint/party/getParty/"
getPartyMatchesBasicUrl: "https://kit.jyllands-posten.dk/electionendpoint/party/getUserMatches/"
getPartyPollResult: "https://kit.jyllands-posten.dk/electionendpoint/party/getPartyResult/"
getPollResultsBasicUrl: "https://kit.jyllands-posten.dk/electionendpoint/poll/calculateResult/"
getQuestionsBasicUrl: "https://kit.jyllands-posten.dk/electionendpoint/poll/getQuestions/"
    """

'\ncompareCandidateUserUrl: "https://kit.jyllands-posten.dk/electionendpoint/candidate/getUserResult/"\ncomparePartyUserUrl: "https://kit.jyllands-posten.dk/electionendpoint/party/getUserResult/"\ndefaultDistrictID: "49"\nelectionsId: "28"\ngetCandidateBasicUrl: "https://kit.jyllands-posten.dk/electionendpoint/candidate/getCandidate/"\ngetCandidateMatchesBasicUrl: "https://kit.jyllands-posten.dk/electionendpoint/candidate/getUserMatches/"\ngetCandidateMatchesWithCandidateUrl: "https://kit.jyllands-posten.dk/electionendpoint/candidate/getCandidateMatches/"\ngetCandidateMatchesWithPartyUrl: "https://kit.jyllands-posten.dk/electionendpoint/party/getCandidateMatches/"\ngetCandidatePollResult: "https://kit.jyllands-posten.dk/electionendpoint/candidate/getCandidateResult/"\ngetCandidatesByDistrictBasicUrl: "https://kit.jyllands-posten.dk/electionendpoint/district/getCandidates/"\ngetCandidatesByPartyAndDistrictUrl: "https://kit.jyllands-posten.dk/electionendpoint/candidate/getCandidates/"\ng

In [23]:
jp_root_url = "https://kit.jyllands-posten.dk/electionendpoint/"  #district/getDistricts/28

In [24]:
# hev rå data ud af tv2s og konverterede manuelt til en json
storkredse = json.load(open("raw_data/TV2/greaterConstituencies.json"))

In [3]:
# bemærkede at tv hentede svarende på den her måde
svar = {}
for storkreds in storkredse:
    print(storkreds['value'], storkreds['label'])
    res = get(f"https://election-api.services.tv2.dk/fv/fv22/candidatetest/answers/{storkreds['value']}")
    svar[storkreds['label']] = res.json()
#gem svarerne
json.dump(svar, open('raw_data/TV2/alle_svar.json', 'w'))

9000400 Bornholms Storkreds
9000600 Fyns Storkreds
9000200 Københavns Omegns Storkreds
9000100 Københavns Storkreds
9001000 Nordjyllands Storkreds
9000300 Nordsjællands Storkreds
9000500 Sjællands Storkreds
9000700 Sydjyllands Storkreds
9000900 Vestjyllands Storkreds
9000800 Østjyllands Storkreds


In [25]:
#svar = json.load(open('raw_data/TV2/alle_svar.json'))

In [26]:
# pt ignorere vi "regionale spørgsmål" fordi de i sagens natur kun er besvaret af et mindretal af kandidaterne...måske behøver jeg ikke gøre det...
tv2_spg = ["1a", "1b", "2a", "2b", "3a", "3b", "4a", "4b", "5a", "5b", "6a", "6b", "7a", "7b", "8a", "8b", "9a", "9b", "10a", "10b", "11a", "11b", "12a", "12b"]

In [27]:
# smæk det hele pænt sammen

tv2_svar = {}
tv2_kandidater = {}
for storkreds, kandidater in svar.items():
    for kandidat in kandidater:
        tv2_kandidater[kandidat['id']] = {'navn': kandidat['name'], 'storkreds': kandidat['areaName'], 'parti': kandidat['partyName'], 'alder': kandidat['age']}
        tv2_svar[kandidat['id']] = {x: kandidat['answers'][x]['answer'] for x in tv2_spg if x in kandidat['answers']}

In [28]:
tv2_kandidater = pd.DataFrame(tv2_kandidater).T
tv2_svar = pd.DataFrame(tv2_svar).T

In [29]:
# norm mellem 0 og 1
ma = tv2_svar.max()
mi = tv2_svar.min()
tv2_svar = (tv2_svar-mi)/(ma-mi)

In [30]:
q = tv2_svar.copy()
q['parti'] = [tv2_kandidater.loc[x, 'parti'] for x in tv2_svar.index]
q['navn'] = [tv2_kandidater.loc[x, 'navn'] for x in tv2_svar.index]
q['storkreds'] = [tv2_kandidater.loc[x, 'storkreds'] for x in tv2_svar.index]
q['alder'] = [tv2_kandidater.loc[x, 'alder'] for x in tv2_svar.index]
q.parti.replace("Konservative", "Det Konservative Folkeparti", inplace=True)

In [31]:
# DR's data er...mere gemt...men jeg fandt en acceptabel måde at hive det ud på via scrapy

In [32]:
objects = []
with (open("TakeTheDR/outputs.pickle", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [33]:
pd.DataFrame([[len(y) for y in x['kandidat']] for x in objects])[0].value_counts()

25    882
0     114
Name: 0, dtype: int64

In [34]:
# 882 har svaret.

In [35]:
# Vi beholder kun dem der har svaret.
objects = [x['kandidat'] for x in objects if x['kandidat'][0]]

In [36]:
sum(pd.Series([pd.DataFrame(x[0])['Answer'].sum() for x in objects])==0)
# hmm og 7 har enten svaret total nej til alt eller har ikke rigtigt svaret

7

In [37]:
objects = [x for x in objects if pd.DataFrame(x[0])['Answer'].sum() != 0]

In [38]:
len(objects)

875

In [45]:
bogstaver = json.load(open('various.json'))['bogstav_leg']

In [46]:
dr_data = []
for i, dd in enumerate(objects):
    person = {'navn': " ".join([dd[1]['Firstname'], dd[1]['LastName']]), 'parti': bogstaver[dd[1]['CurrentPartyCode']]}
    svar = (pd.DataFrame(dd[0])[['QuestionID', 'Answer']].set_index('QuestionID').T-1)/4
    svar.columns = svar.columns.astype('str')
    svar['navn'] = person['navn']
    svar['parti'] = person['parti']
    svar.index = [f'DR{i}']
    dr_data.append(svar)
dr_df = pd.concat(dr_data)
dr_df = dr_df.replace(-0.25, np.nan)

In [50]:
q[q.navn.str.contains('Carøe')]

,1a,1b,2a,2b,3a,3b,4a,4b,5a,5b,...,10a,10b,11a,11b,12a,12b,parti,navn,storkreds,alder
16040,0.75,0.75,1.0,0.75,0.0,1.0,1.0,1.0,0.0,0.75,...,0.25,1.0,1.0,0.0,0.5,0.5,Socialistisk Folkeparti,Astrid Carøe Rasmussen,Sjællands Storkreds,28


In [49]:
dr_df[dr_df.navn.str.contains('Carøe')]

QuestionID,530,531,533,534,535,537,538,540,541,543,...,552,553,555,556,557,559,561,563,navn,parti
DR679,1.0,1.0,0.0,0.0,0.75,0.0,0.75,0.0,1.0,0.0,...,0.75,0.75,0.0,0.0,1.0,1.0,1.0,1.0,Astrid Carøe,Socialistisk Folkeparti


In [51]:
all_data = dr_df.merge(q, on=['navn', 'parti'])

In [68]:
alle_navne = pd.concat([dr_df.navn, q.navn])
folk_mangler = alle_navne[~alle_navne.isin(all_data.navn)]
len(folk_mangler)

265

In [69]:
# 265 mangler...kan være fordi de kun har svaret på 1 test. Kan være fordi deres navn er forskelligt i de 2 tests
# Lets find out =)

In [118]:
flatten=lambda l: sum(map(flatten,l),[]) if isinstance(l,list) else [l]
potids = {}
noid = []
for person in folk_mangler:
    navne = person.split()
    potid = []
    for navn in navne:
        liid = folk_mangler[folk_mangler.str.contains(navn)].to_list()
        if len(liid) == 2:
            potid.append(liid)
    potid = list(set(flatten(potid)))
    if potid:
        potid.remove(person)
        if len(potid) == 1:
            potids[person] = potid[0]
        else:
            noid.append(person)
    else:
        noid.append(person)
potids2 = {}
for k, v in potids.items():
    if v not in potids2:
        potids2[k] = v

In [121]:
manuel_data = {
'Carsten Damgaard':'Carsten Damgaard Møller',
'Cecilie Winther Kristensen': 'Cecillie Winther Kristensen',
'Christian Jacobsen':'Christian Ivan Jacobsen',
'Lars Bregnbak':'Lars Bregbak',
'Lars Holmqvist': 'Lars Høyer Holmqvist',
'Mette Kierkgaard': 'Mette Kierkegaard',
'Rene Danielsson':'René Danielsson',
'Theresa Berg':'Theresa Berg Andersen',
'Sikander Siddique': 'Sikandar Siddique',
 'Chresten Heesgård Ibsen': 'Chresten H. Ibsen',
 'Rasmus Bøgh Vinter': 'Rasmus Bøgh Vinther',
 'Mette Bram': 'Mette Kaasfeldt Bram',
 'Louise Louring': 'Louise Lingren Louring',
 'Niko Grunfeld': 'Niko Grünfeld',
 'Mohammed Abdikarim': 'Mohamed Abdikarim',
 'Christina Olumeko': 'Christina Sade Olumeko',
 'Mette Bloch': 'Mette Bloch Hansen',
 'Michael Nedersøe': 'Michael Kim Bladt Nedersøe',
 'Anders Vistisen': 'Anders Primdahl Vistisen',
 'Palle Rosenkilde': 'Palle Bjørn Rosenkilde',
 'David Rønne': 'David Schack Trappaud Rønne',
 'Brian Bruun': 'Brian Borglund Bruun',
 'Marina Andersen': 'Marina Fløjgaard Andersen',
 'Jon Stephensen': 'Jon Læssøe Stephensen',
 'Isabella Arendt': 'Isabella Arendt Vlasman',
 'Allan Bilgaard Beier': 'Allan Beier',
 'Peter Zinckernagel': 'Peter Kjølbye Zinckernagel',
 'Ida Auken': 'Ida Margrete Meier Auken',
 'Bjarne Hartung Kirkegaard': 'Bjarne H. Kirkegaard',
 'Diana-Ester Brandt Eriksen': 'Diana-Ester Eriksen',
 'Martin Kjærulf': 'Martin Kjærulff',
 'Kasper Krebs': 'Kasper Sten Krebs',
 'Anna Astrid Hansen Calero': 'Anna Astrid Hansen calero',
 'Victoria Velasquez': 'Victoria Velásquez',
 'Lars Chr. Lilleholt': 'Lars Christian Lilleholt',
 'Thomas Holbech Anker': 'Thomas Anker',
 'Jakob Kjærsgaard': 'Jakob Kjærsgaard Mortensen',
 'Hans Christian Hidalgo Wullf': 'Hans Christian Hidalgo Wulff',
 'Mikkel Dencker': 'Mikkel S. Dencker',
 'Jonas Hulgaard  Ammundsen': 'Jonas Hulgaard Ammundsen',
 'Anastasia Valentin Rasmussen': 'Anastasia Valentin Mikitsiuk Rasmussen',
 'Jørgen Juhl': 'Jørgen Mathow-Juhl',
 'Henrik\xa0Tørnqvist Lauridsen': 'Henrik Tørnqvist Lauridsen',
 'Niels Grønlund': 'Niels Ture Holm Grønlund',
 'Bjørn Bøje Brandenborg': 'Bjørn Brandenborg',
 'Lone Langballe': 'Lone Vase Langballe',
 'Hans Fonsbøl': 'Hans Biering Fonsbøl',
 'Rasmus Vestergaard': 'Rasmus Vestergaard Madsen',
 'Frida Vyff Bruun\xa0':'Frida Vyff Bruun',
 'Karin B Nissen': 'Karin B. Nissen',
 'Marianne Mørk Mathiessen': 'Marianne Mørk Mathiesen',
 'Line Gessø Hansen': 'Line Gessø',
 'Stén Knuth': 'Sten Knuth',
 'Marcus Danielsson': 'Marcus Techow Danielsson',
 'Yadvinder Singh Sekhon': 'Yadvinder Singh',
 'Merete Dea Larsen': 'Merete Dea Møller Larsen',
 'Nicolai Sandager': 'Nicolai Wøldike Sandager',
 'Mads Bruun Buch Olsen': 'Mads Olsen',
 'Jeanne Rygas Bergmansen': 'Jeanne Bergmansen',
 'Astrid Carøe': 'Astrid Carøe Rasmussen',
 'Githa Nelander': 'Githa Lone Nelander',
 'Kaare Dybvad': 'Kaare Dybvad Bek',
 'Sidsel Dyekjær': 'Sidsel Marie Dyekjær',
 'Nicklas Gjedsig': 'Nicklas Gjedsig Larsen',
 'Mikkel Larsson': 'Mikkel Høj Bagger Larsson',
 'Joachim Kjeldsen': 'Joachim Plaetner Kjeldsen',
 'H.P. Beck': 'Hans Peter Beck',
 'Benjamin Jørgensen': 'Benjamin Risager Jørgensen',
 'Lars Ole Skovgaard': 'Lars Ole Skovgaard Larsen',
 'Marianne Lynghøj': 'Marianne Lynghøj Pedersen',
 'Kim Mejnert Frydensberg': 'Kim Mejnert Frydensbjerg',
 'Britta Glønborg Kristensen': 'Britta Kristensen',
 'Egil Fabritius Hulgaard': 'Egil Hulgaard', 
 'Simon Lavdal': 'Simon Lavdal-Pedersen',
 'Kenneth Werby': 'Kenneth Werby Jensen',
 'Magnus Heunicke': 'Magnus Johannes Heunicke',
 'Dorte Fløjgaard': 'Marina Fløjgaard Andersen',
 'Henning Østergaard': 'Henning Østergaard-Christensen'}

In [124]:
q.navn = q.navn.apply(lambda x: manuel_data[x] if x in manuel_data else x)
dr_df.navn = dr_df.navn.apply(lambda x: manuel_data[x] if x in manuel_data else x)
all_data = dr_df.merge(q, on=['navn', 'parti'])

In [125]:
alle_navne = pd.concat([dr_df.navn, q.navn])
folk_mangler = alle_navne[~alle_navne.isin(all_data.navn)]
len(folk_mangler)

110

In [27]:
# ok...110 ser ud til ikke at havde svaret...bl.a. Nikolaj Wammen

In [126]:
# udfyld manglende svar med partiets gennemsnit
dk_spg = ['530', '531', '533', '534', '535', '537', '538', '540', '541', '543','544', '545', '546', '547', '548', '550', '551', '552', '553', '555',
       '556', '557', '559', '561', '563', '1a', '1b', '2a','2b', '3a', '3b', '4a', '4b', '5a', '5b', '6a', '6b', '7a', '7b', '8a',
       '8b', '9a', '9b', '10a', '10b', '11a', '11b', '12a', '12b']
parti_linjen = round(all_data.groupby('parti')[dk_spg].mean()*4)/4  # gennemsnitssvaret per parti, rundet til 1-5 (0-4) svarmulighederne

def fald_ind(x):
    x.loc[x[x.isna()].index] = parti_linjen.loc[x.parti, x[x.isna()].index]
    return x
    
all_data_final = all_data.apply(fald_ind, axis=1)

In [127]:
all_data_final.to_feather("2022_Lasse_data.feather")